In [7]:
import requests
import pandas as pd

token = "eyJvcmciOiI1ZTU1NGUxOTI3NGE5NjAwMDEyYTNlYjEiLCJpZCI6IjcxZGFmYmI1NGZkYTQ4NjI5ZGU0Mjk5OTM1ZDlmMzdmIiwiaCI6Im11cm11cjEyOCJ9"
headers = {"Authorization": token}

request = requests.get(url='https://api.dataplatform.knmi.nl/edr/collections/observations/instances/unvalidated/locations/06201?datetime=2023-11-01T04%3A10%3A00Z%2F2023-11-03T04%3A10%3A00Z&parameter-name=t_dryb_10', headers=headers)
jsonre = request.json()
jsonre

r = requests.get(f"https://api.dataplatform.knmi.nl/edr/collections/observations", headers=headers)
r.raise_for_status()
r.json()

{'detail': 'Maximum query size exceeded. Your query has a score of 7524, the maximum score is 1500. Your query score is calculated as: 7524 * 1 * 1 (total_days * parameter_count * station_count). You requested 7524 days of data, the maximum is 366. Please reduce the number of stations or parameters, or the time range. The maximum score allows a query of a year of data for four parameters for a single station. Reducing the time range allows more parameters.'}

In [100]:
token = "eyJvcmciOiI1ZTU1NGUxOTI3NGE5NjAwMDEyYTNlYjEiLCJpZCI6IjcxZGFmYmI1NGZkYTQ4NjI5ZGU0Mjk5OTM1ZDlmMzdmIiwiaCI6Im11cm11cjEyOCJ9"
headers = {"Authorization": token}

request = requests.get(url='https://api.dataplatform.knmi.nl/edr/collections/observations/instances/unvalidated/locations', headers=headers)
jsonre = request.json()   

df = pd.DataFrame(columns=['Naam', 'id'])
length = len(jsonre['features'])

count = 0 
while count != length:
    df.loc[len(df)] = [jsonre['features'][count]['properties']['name'], jsonre['features'][count]['id']]
    count += 1
    
df['Naam'] = df['Naam'].str.lower()

,Naam,id
47,vlissingen aws,06310


In [128]:
from datetime import date, timedelta
import pandas as pd
weerstat = pd.read_csv('weerstations6.csv',sep=',')
weerstat['station_id'] = '0'
weerstat['date'] = date(year=2023, month=1, day=1)
weerstat['NAME'] = weerstat['NAME'].str.lower()

for i in weerstat.index:
    for j in df.index:
        if weerstat.loc[i, 'NAME'] in df.loc[j, 'Naam']:
            weerstat.loc[i, 'station_id'] = df.loc[j, 'id']
        if weerstat['NAME'][i] == 'gilze-rijen':
            weerstat.loc[i, 'station_id'] = '06350'
        if weerstat['NAME'][i] == 'cabauw mast':
            weerstat.loc[i, 'station_id'] = '06348'
        if weerstat['NAME'][i] == 'valkenburg zh':
            weerstat.loc[i, 'station_id'] = '06210'
        if weerstat['NAME'][i] == 'hoorn terschelling':
            weerstat.loc[i, 'station_id'] = '06251'
        if weerstat['NAME'][i] == 'tholen':
            weerstat.loc[i, 'station_id'] = '06310'
        if weerstat['NAME'][i] == 'hoofdplaat':
            weerstat.loc[i, 'station_id'] = '06310'
        if weerstat['NAME'][i] == 'schaar':
            weerstat.loc[i, 'station_id'] = '06310'          

In [129]:
def api_prepper(station_id, start_date, end_date):
    api = f'https://api.dataplatform.knmi.nl/edr/collections/observations/instances/unvalidated/locations/{station_id}?datetime={start_date}T04%3A10%3A00Z%2F{end_date}T04%3A10%3A00Z&parameter-name=t_dryb_10'
    return api

api_prepper('05554', '2023-11-3', '2023-11-4')

def date_adderv2(df, num_days):
    count = 0
    copy = df.copy()
    while num_days != count:
        copy['date'] = copy['date'] + timedelta(days=1)
        df = pd.concat([df, copy], ignore_index=True)
        count += 1
    return df

weather_stats = date_adderv2(weerstat, 364)
weather_stats

,WKT,fid,LON(east),LAT(north),ALT(m),NAME,geometry,STN_x,STN_y,station_id,date
0,MULTIPOLYGON (((3.74129852482596 51.2716446632...,1,3.672,51.379,0.0,hoofdplaat,MULTIPOLYGON (((3.74129852482596 51.2716446632...,311,311,06310,2023-01-01
1,MULTIPOLYGON (((3.52181453257886 51.2888242988...,2,3.379,51.381,0.0,cadzand,MULTIPOLYGON (((3.52181453257886 51.2888242988...,308,308,06308,2023-01-01
2,MULTIPOLYGON (((4.06878932447641 51.2500208105...,3,3.861,51.226,1.7,westdorpe,MULTIPOLYGON (((4.06878932447641 51.2500208105...,319,319,06319,2023-01-01
3,MULTIPOLYGON (((4.24560421785982 51.3757819124...,4,4.193,51.480,0.0,tholen,MULTIPOLYGON (((4.24560421785982 51.3757819124...,331,331,06310,2023-01-01
4,MULTIPOLYGON (((4.27244377929923 51.3767449421...,5,4.342,51.449,19.2,woensdrecht,MULTIPOLYGON (((4.27244377929923 51.3767449421...,340,340,06340,2023-01-01
...,...,...,...,...,...,...,...,...,...,...,...
18245,MULTIPOLYGON (((5.70024659260853 53.4733514095...,46,5.752,53.224,1.2,leeuwarden,MULTIPOLYGON (((5.70024659260853 53.4733514095...,270,270,06270,2023-12-31
18246,MULTIPOLYGON (((5.936073908364 53.483281002826...,47,6.200,53.413,2.9,lauwersoog,MULTIPOLYGON (((5.936073908364 53.483281002826...,277,277,06277,2023-12-31
18247,MULTIPOLYGON (((4.75766177320479 53.1274967578...,48,4.921,53.241,10.8,vlieland,MULTIPOLYGON (((4.75766177320479 53.1274967578...,242,242,06242,2023-12-31
18248,MULTIPOLYGON (((5.14868408460081 53.3831998808...,49,5.346,53.392,0.7,hoorn terschelling,MULTIPOLYGON (((5.14868408460081 53.3831998808...,251,251,06251,2023-12-31


In [131]:
count = 0
while count != 50:
    url = api_prepper(weather_stats['station_id'][count], str(weather_stats['date'].min()), str(weather_stats['date'].max()))
    count += 1
    print(url)
    

https://api.dataplatform.knmi.nl/edr/collections/observations/instances/unvalidated/locations/06310?datetime=2023-01-01T04%3A10%3A00Z%2F2023-12-31T04%3A10%3A00Z&parameter-name=t_dryb_10
https://api.dataplatform.knmi.nl/edr/collections/observations/instances/unvalidated/locations/06308?datetime=2023-01-01T04%3A10%3A00Z%2F2023-12-31T04%3A10%3A00Z&parameter-name=t_dryb_10
https://api.dataplatform.knmi.nl/edr/collections/observations/instances/unvalidated/locations/06319?datetime=2023-01-01T04%3A10%3A00Z%2F2023-12-31T04%3A10%3A00Z&parameter-name=t_dryb_10
https://api.dataplatform.knmi.nl/edr/collections/observations/instances/unvalidated/locations/06310?datetime=2023-01-01T04%3A10%3A00Z%2F2023-12-31T04%3A10%3A00Z&parameter-name=t_dryb_10
https://api.dataplatform.knmi.nl/edr/collections/observations/instances/unvalidated/locations/06340?datetime=2023-01-01T04%3A10%3A00Z%2F2023-12-31T04%3A10%3A00Z&parameter-name=t_dryb_10
https://api.dataplatform.knmi.nl/edr/collections/observations/instance